# Tutorial A5 (Breaking AES-256 Bootloader)

This tutorial will take you through a complete attack on an encrypted bootloader using AES-256. This demonstrates how to use side-channel power analysis on practical systems, along with discussing how to perform analysis with different Analyzer models.

## Background

In the world of microcontrollers, a bootloader is a special piece of firmware that is made to let the user upload new programs into memory. This is especially useful for devices with complex code that may need to be patched or otherwise updated in the future - a bootloader makes it possible for the user to upload a patched version of the firmware onto the micro. The bootloader receives information from a communication line (a USB port, serial port, ethernet port, WiFi connection, etc...) and stores this data into program memory. Once the full firmware has been received, the micro can happily run its updated code.

There is one big security issue to worry about with bootloaders. A company may want to stop their customers from writing their own firmware and uploading it onto the micro. For example, this might be for protection reasons - hackers might be able to access parts of the device that weren't meant to be accessed. One way of stopping this is to add encryption. The company can add their own secret signature to the firmware code and encrypt it with a secret key. Then, the bootloader can decrypt the incoming firmware and confirm that the incoming firmware is correctly signed. Users will not know the secret key or the signature tied to the firmware, so they won't be able to "fake" their own.

This tutorial will work with a simple AES-256 bootloader. The victim will receive data through a serial connection, decrypt the command, and confirm that the included signature is correct. Then, it will only save the code into memory if the signature check succeeded. To make this system more robust against attacks, the bootloader will use cipher-block chaining (CBC mode). Our goal is to find the secret key and the CBC initialization vector so that we could successfully fake our own firmware.

### Bootloader Communications Protocol

The bootloader's communications protocol operates over a serial port at 38400 baud rate. The bootloader is always waiting for new data to be sent in this example; in real life one would typically force the bootloader to enter through a command sequence.

Commands sent to the bootloader look as follows:

```
       |<-------- Encrypted block (16 bytes) ---------->|
       |                                                |
+------+------+------+------+------+------+ .... +------+------+------+
| 0x00 |    Signature (4 Bytes)    |  Data (12 Bytes)   |   CRC-16    |
+------+------+------+------+------+------+ .... +------+------+------+
```

This frame has four parts:

* `0x00`: 1 byte of fixed header
* Signature: A secret 4 byte constant. The bootloader will confirm that this signature is correct after decrypting the frame.
* Data: 12 bytes of the incoming firmware. This system forces us to send the code 12 bytes at a time; more complete bootloaders may allow longer variable-length frames.
* CRC-16: A 16-bit checksum using the CRC-CCITT polynomial (0x1021). The LSB of the CRC is sent first, followed by the MSB. The bootloader will reply over the serial port, describing whether or not this CRC check was valid.

As described in the diagram, the 16 byte block is not sent as plaintext. Instead, it is encrypted using AES-256 in CBC mode. This encryption method will be described in the next section.

The bootloader responds to each command with a single byte indicating if the CRC-16 was OK or not:

```
            +------+
CRC-OK:     | 0xA1 |
            +------+

            +------+
CRC Failed: | 0xA4 |
            +------+
```
Then, after replying to the command, the bootloader veries that the signature is correct. If it matches the expected manufacturer's signature, the 12 bytes of data will be written to flash memory. Otherwise, the data is discarded.

### Details of AES-256 CBC

The system uses the AES algorithm in Cipher Block Chaining (CBC) mode. In general one avoids using encryption 'as-is' (i.e. Electronic Code Book), since it means any piece of plaintext always maps to the same piece of ciphertext. Cipher Block Chaining ensures that if you encrypted the same thing a bunch of times it would always encrypt to a new piece of ciphertext.

You can see another reference on the design of the encryption side; we'll be only talking about the decryption side here. In this case AES-256 CBC mode is used as follows, where the details of the AES-256 Decryption block will be discussed in detail later:

![AES-256](https://wiki.newae.com/images/8/88/Aes256_cbc.png)

This diagram shows that the output of the decryption is no longer used directly as the plaintext. Instead, the output is XORed with a 16 byte mask, which is usually taken from the previous ciphertext. Also, the first decryption block has no previous ciphertext to use, so a secret initialization vector (IV) is used instead. If we are going to decrypt the entire ciphertext (including block 0) or correctly generate our own ciphertext, we'll need to find this IV along with the AES key.

### Attacking AES-256

The system in this tutorial uses AES-256 encryption, which has a 256 bit (32 byte) key - twice as large as the 16 byte key we've attacked in previous tutorials. This means that our regular AES-128 CPA attacks won't quite work. However, extending these attacks to AES-256 is fairly straightforward: the theory is explained in detail in Extending AES-128 Attacks to AES-256.

As the theory page explains, our AES-256 attack will have 4 steps:

1. Perform a standard attack (as in AES-128 decryption) to determine the first 16 bytes of the key, corresponding to the 14th round encryption key.
1. Using the known 14th round key, calculate the hypothetical outputs of each S-Box from the 13th round using the ciphertext processed by the 14th round, and determine the 16 bytes of the 13th round key manipulated by inverse MixColumns.
1. Perform the MixColumns and ShiftRows operation on the hypothetical key determined above, recovering the 13th round key.
1. Using the AES-256 key schedule, reverse the 13th and 14th round keys to determine the original AES-256 encryption key.

## Firmware

For this tutorial, we'll be using the `bootloader-aes256` project, which we'll build as usual:

In [393]:
PLATFORM = "CWLITEARM"
CRYPTO_TARGET="NONE"

In [461]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../hardware/victims/firmware/bootloader-aes256
make PLATFORM=$1 CRYPTO_TARGET=$2

rm -f -- bootloader-aes256-CWLITEARM.hex
rm -f -- bootloader-aes256-CWLITEARM.eep
rm -f -- bootloader-aes256-CWLITEARM.cof
rm -f -- bootloader-aes256-CWLITEARM.elf
rm -f -- bootloader-aes256-CWLITEARM.map
rm -f -- bootloader-aes256-CWLITEARM.sym
rm -f -- bootloader-aes256-CWLITEARM.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- bootloader.s aes256.s crcccitt.s simpleserial.s stm32f3_hal.s stm32f3_hal_lowlevel.s stm32f3_sysmem.s
rm -f -- bootloader.d aes256.d crcccitt.d simpleserial.d stm32f3_hal.d stm32f3_hal_lowlevel.d stm32f3_sysmem.d
rm -f -- bootloader.i aes256.i crcccitt.i simpleserial.i stm32f3_hal.i stm32f3_hal_lowlevel.i stm32f3_sysmem.i
.
-------- begin --------
arm-none-eabi-gcc (15:6.3.1+svn253039-1build1) 6.3.1 20170620
Copyright (C) 2016 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling C: bootloader.c
arm-none-eabi-gcc -c

## Capturing Traces

### Setup

To start, we'll proceed with setup as usual:

In [153]:
%run "Helper Scripts/CWLite_Connect.ipynb"

In [154]:
%run "Helper Scripts/Setup_Target_Generic.ipynb"

In [387]:
# uncomment based on your target
fw_path = "../../hardware/victims/firmware/bootloader-aes256/bootloader-aes256-CWLITEARM.hex"
#%run "Helper Scripts/Program_XMEGA.ipynb"
%run "Helper Scripts/Program_STM.ipynb"
#%run "Helper Scripts/No_Programmer.ipynb"

In [462]:
program_target(scope, fw_path)

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to programming 5919 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 5919 bytes


### Calculating the CRC

The next step we'll need to take in attacking this target is to communicate with it. Most of the transmission is fairly straight forward, but the CRC is a little tricky. Luckily, there's a lot of open source out there for calculating CRCs. In this case, we'll pull some code from pycrc:

In [157]:
# Class Crc
#############################################################
# These CRC routines are copy-pasted from pycrc, which are:
# Copyright (c) 2006-2013 Thomas Pircher <tehpeh@gmx.net>
#
class Crc(object):
    """
    A base class for CRC routines.
    """

    def __init__(self, width, poly):
        """The Crc constructor.

        The parameters are as follows:
            width
            poly
            reflect_in
            xor_in
            reflect_out
            xor_out
        """
        self.Width = width
        self.Poly = poly


        self.MSB_Mask = 0x1 << (self.Width - 1)
        self.Mask = ((self.MSB_Mask - 1) << 1) | 1

        self.XorIn = 0x0000
        self.XorOut = 0x0000

        self.DirectInit = self.XorIn
        self.NonDirectInit = self.__get_nondirect_init(self.XorIn)
        if self.Width < 8:
            self.CrcShift = 8 - self.Width
        else:
            self.CrcShift = 0

    def __get_nondirect_init(self, init):
        """
        return the non-direct init if the direct algorithm has been selected.
        """
        crc = init
        for i in range(self.Width):
            bit = crc & 0x01
            if bit:
                crc ^= self.Poly
            crc >>= 1
            if bit:
                crc |= self.MSB_Mask
        return crc & self.Mask


    def bit_by_bit(self, in_data):
        """
        Classic simple and slow CRC implementation.  This function iterates bit
        by bit over the augmented input message and returns the calculated CRC
        value at the end.
        """
        # If the input data is a string, convert to bytes.
        if isinstance(in_data, str):
            in_data = [ord(c) for c in in_data]

        register = self.NonDirectInit
        for octet in in_data:
            for i in range(8):
                topbit = register & self.MSB_Mask
                register = ((register << 1) & self.Mask) | ((octet >> (7 - i)) & 0x01)
                if topbit:
                    register ^= self.Poly

        for i in range(self.Width):
            topbit = register & self.MSB_Mask
            register = ((register << 1) & self.Mask)
            if topbit:
                register ^= self.Poly

        return register ^ self.XorOut
    
bl_crc = Crc(width = 16, poly=0x1021)

Now we can easily get the CRC for our message by calling `bl_crc.bit_by_bit(message)`. 

### Communicating with the Bootloader

With that done, we can start communicating with the bootloader. Recall that the bootloader expects:
* To start with `0x00`
* A 16 byte encrypted message (4 bytes signature + 12 bytes data)
* CRC16

We don't really care what the 16 byte message is (just that each is different so that we get a variety of hamming weights), so we'll use the same text/key module from earlier attacks.

We can now run the following block, and we should get `0xA4` back. You may need to run this block a few times to get the right response back.

In [170]:
from chipwhisperer.capture.acq_patterns.basic import AcqKeyTextPattern_Basic
import time
message = [0x00]
ktp = AcqKeyTextPattern_Basic(target=target)

# clear serial buffer
num_char = target.ser.inWaiting()
print(target.ser.read(num_char))

key, text = ktp.newPair() #don't care about key here
message.extend(text)

crc = bl_crc.bit_by_bit(text)

message.append(crc >> 8)
message.append(crc & 0xFF)

target.ser.write(message)
time.sleep(0.1)

num_char = target.ser.inWaiting()
response = target.ser.read(num_char)
print("Response: {:02X}".format(ord(response[0])))

¡¡
Response: A4


### Capturing Traces

With that out of the way, we can proceed to capturing our traces. The normal 5000 traces we capture isn't long enough to get the rounds we care about, so we'll need to increase it (11000 should be fine):

In [102]:
scope.adc.samples = 11000

We'll be working with Analyzer, so we'll need to use a ChipWhisperer project to store our traces and text:

In [23]:
from chipwhisperer.common.api.ProjectFormat import ProjectFormat
project = cw.createProject("projects/Tutorial A5", overwrite=True)
tc = project.getTraceFormat()
ktp = AcqKeyTextPattern_Basic(target=target)

Below you'll find our capture loop. This will be pretty similar to Tutorial B5, but we've added our communication code. We also check the response and just skip the data we get if it isn't correct.

In [24]:
#Capture Traces
from tqdm import tqdm
from chipwhisperer.capture.acq_patterns.basic import AcqKeyTextPattern_Basic
import numpy as np
import time
keys = []
N = 100  # Number of traces
target.init()
for i in tqdm(range(N), desc='Capturing traces'):
    message = [0x00]
    
    num_char = target.ser.inWaiting()
    target.ser.read(num_char)
    
    key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here
    keys.append(key)
    
    message.extend(text)
    
    crc = bl_crc.bit_by_bit(text)
    message.append(crc >> 8)
    message.append(crc & 0xFF)

    # run aux stuff that should run before the scope arms here

    scope.arm()

    # run aux stuff that should run after the scope arms here

    target.ser.write(message)
    timeout = 50
    # wait for target to finish
    while target.isDone() is False and timeout:
        timeout -= 1
        time.sleep(0.01)

    try:
        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition')
    except IOError as e:
        print('IOError: %s' % str(e))

    # run aux stuff that should happen after trace here
    num_char = target.ser.inWaiting()
    response = target.ser.read(num_char)
    if ord(response[0]) != 0xA4:
        # Bad response, just skip
        print("Bad response: {:02X}".format(ord(response[0])))
        continue
    
    tc.addTrace(scope.getLastTrace(), text, "", key)
    
tc._isloaded = True
project.traceManager().appendSegment(tc)

Capturing traces: 100%|██████████| 100/100 [00:12<00:00,  7.71it/s]


With that, we're done with capturing traces! We can now disconnect from the hardware:

In [ ]:
scope.dis()
target.dis()

## Analysis

Now that we have our traces, we can go ahead and perform the attack. As described in the background theory, we'll have to do two attacks - one to get the 14th round key, and another (using the first result) to get the 13th round key. Then, we'll do some post-processing to finally get the 256 bit encryption key.

### 14th Round Key

We can attack the 14th round key with a standard, no-frills CPA attack (using the inverse sbox, since it's a decryption that we're breaking):

In [25]:
import chipwhisperer as cw
from chipwhisperer.analyzer.attacks.cpa import CPA
from chipwhisperer.analyzer.attacks.cpa_algorithms.progressive import CPAProgressive
from chipwhisperer.analyzer.attacks.models.AES128_8bit import AES128_8bit, InvSBox_output

tm = project.traceManager()

attack = CPA()
leak_model = AES128_8bit(InvSBox_output)
attack.setAnalysisAlgorithm(CPAProgressive, leak_model)
attack.setTraceSource(tm)
attack.setTraceStart(0)
attack.setTracesPerAttack(tm.numTraces())
attack.setIterations(1)
attack.setReportingInterval(10)
attack.setTargetSubkeys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])

With the setup done, we can actually preform the attack. 11000 samples is a rather large amount to chew through, so if you want a faster attack you can use a smaller range in `attack.setPointRange()`. `(2900, 4200)` will work for XMEGA, while `(1400, 2600)` will work for the STM32F3 (CWLite ARM).

Below you'll find the key that we should recover from this attack. You may want to check what we actually get against this key to make sure the attack is working.

In [29]:
key = [0xea, 0x79, 0x79, 0x20, 0xc8, 0x71, 0x44, 0x7d, 0x46, 0x62, 0x5f, 0x51, 0x85, 0xc1, 0x3b, 0xcb]
#key = keys[0]

In [30]:
import pandas as pd
def format_stat(stat):
    return str("{:02X}<br>{:.3f}".format(stat[0], stat[2]))

def color_corr_key(row):
    global key
    ret = [""] * 16
    for i,bnum in enumerate(row):
        if bnum[0] == key[i]:
            ret[i] = "color: red"
        else:
            ret[i] = ""
    return ret

from IPython.display import clear_output
import numpy as np
        
def stats_callback():
    attack_results = attack.getStatistics()
    attack_results.setKnownkey(key)
    stat_data = attack_results.findMaximums()
    df = pd.DataFrame(stat_data).transpose()
    clear_output(wait=True)
    display(df.head().style.format(format_stat).apply(color_corr_key,axis=1))
    
attack.setPointRange((0, -1))
attack_results = attack.processTracesNoGUI(stats_callback)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,EA0.880,790.831,790.883,200.894,C80.869,710.883,440.861,7D0.870,460.905,620.878,5F0.860,510.872,850.871,C10.838,3B0.867,CB0.879
1,3F0.521,150.487,BA0.488,D00.486,5F0.505,740.497,CF0.513,E90.482,9B0.507,3B0.478,480.482,850.475,CA0.481,A10.513,1A0.512,EC0.477
2,350.487,440.473,C50.471,0C0.481,150.471,A20.476,9C0.493,3C0.467,540.503,120.473,260.462,AF0.465,960.476,AB0.487,0B0.508,E40.460
3,840.483,B10.463,060.471,9C0.478,8C0.468,530.469,580.482,CD0.461,FA0.496,EC0.467,DC0.460,5B0.455,550.471,830.484,990.464,F30.442
4,DB0.472,2D0.460,DB0.460,4C0.477,6B0.466,5A0.468,A90.476,4A0.461,860.484,4E0.459,160.456,C70.454,350.458,E00.476,740.461,910.441


### 13th Round Key

Analyzer doesn't have a leakage model for the 13th round key built in, so we'll need to create our own. An example class is shown below along with the beginning of the setup. **NOTE: You'll need to update `calc_round_key` with the key you found in the last step**

In [31]:
import chipwhisperer as cw
from chipwhisperer.analyzer.attacks.cpa import CPA
from chipwhisperer.analyzer.attacks.cpa_algorithms.progressive import CPAProgressive
from chipwhisperer.analyzer.attacks.models.AES128_8bit import AES128_8bit, AESLeakageHelper
from chipwhisperer.analyzer.preprocessing.resync_sad import ResyncSAD

class AES256_Round13_Model(AESLeakageHelper):
    def leakage(self, pt, ct, guess, bnum):
        #You must put YOUR recovered 14th round key here - this example may not be accurate!
        calc_round_key = [0xea, 0x79, 0x79, 0x20, 0xc8, 0x71, 0x44, 0x7d, 0x46, 0x62, 0x5f, 0x51, 0x85, 0xc1, 0x3b, 0xcb]
        xored = [calc_round_key[i] ^ pt[i] for i in range(0, 16)]
        block = xored
        block = self.inv_shiftrows(block)
        block = self.inv_subbytes(block)
        block = self.inv_mixcolumns(block)
        block = self.inv_shiftrows(block)
        result = block
        return self.inv_sbox((result[bnum] ^ guess[bnum]))
    
attack = CPA()
leak_model = AES128_8bit(AES256_Round13_Model)
attack.setAnalysisAlgorithm(CPAProgressive, leak_model)
attack.setTraceSource(project.traceManager())

#### Resyncing Traces (XMEGA Only)

The traces for the XMEGA version of the firmware become desynced around sample 7000. This is due to a non-constant AES implementation: the code does not always take the same amount of time to run for every input. (It's actually possible to do a timing attack on this AES implementation! We'll stick with our CPA attack for now.)

While this does open up a timing attack, it actually makes our AES attack a little harder, since we'll have to resync the traces. Luckily, this can be done pretty easily by using the ResyncSAD preprocessing module:

In [ ]:
resync_traces = ResyncSAD(tm)
resync_traces.enabled = True
resync_traces.ref_trace = 0
resync_traces.target_window = (9100, 9300)
resync_traces.max_shift = 200
attack.setTraceSource(resync_traces)

#### Running the Attack

Like in the 14th round attack, we can use a smaller range of points to make the attack faster. `(8000,10990)` works well for the XMEGA, while `(6500, 8500)` works well for the STM32F3.

You can run the block below and the correct key should be printed out:

In [32]:
attack.setTraceStart(0)
attack.setTracesPerAttack(tm.numTraces())
attack.setIterations(1)
attack.setReportingInterval(10)
attack.setTargetSubkeys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])
attack.setPointRange((6500,8500))
attack_results = attack.processTracesNoGUI(stats_callback)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,C60.894,BD0.902,4E0.891,500.894,AB0.879,CA0.876,750.894,770.887,790.887,870.889,960.845,CA0.883,1C0.903,7F0.866,C50.902,820.916
1,640.495,510.462,AB0.461,B20.455,E10.439,C70.481,5A0.449,5D0.474,1B0.441,F20.450,EC0.461,A40.457,290.467,390.453,ED0.482,740.465
2,590.450,0E0.455,040.448,5F0.451,000.436,AF0.455,DF0.441,A60.471,B80.437,DF0.445,AF0.455,970.444,670.444,210.429,8F0.481,190.453
3,AB0.445,8E0.434,BB0.445,540.445,780.435,F70.444,0C0.431,510.461,030.437,F90.438,EE0.448,850.428,6A0.441,7B0.421,140.447,E60.449
4,E90.437,160.432,E10.444,460.445,AC0.429,540.434,950.430,FD0.432,710.430,740.435,F30.446,750.417,A50.436,2A0.416,B70.438,FF0.439


This, however, isn't actually the 13th round key. To get the real 13th round key, we'll need to run what we've recovered through a `shiftrows()` and `mixcolumns()` operation:

In [35]:
rec_key2 = []
for bnum in attack_results.findMaximums():
    print("Best Guess = 0x{:02X}, Corr = {}".format(bnum[0][0], bnum[0][2]))
    rec_key2.append(bnum[0][0])

from chipwhisperer.analyzer.attacks.models.aes.funcs import shiftrows,mixcolumns
    
real_key2 = shiftrows(rec_key2)
real_key2 = mixcolumns(real_key2)

print("Recovered:", end="")
for subkey in real_key2:
    print(" {:02X}".format(subkey), end="")
print("")

Best Guess = 0xC6, Corr = 0.8937448929000054
Best Guess = 0xBD, Corr = 0.9019453664779397
Best Guess = 0x4E, Corr = 0.891451634170113
Best Guess = 0x50, Corr = 0.8943159714733331
Best Guess = 0xAB, Corr = 0.879475780636866
Best Guess = 0xCA, Corr = 0.8764486807809474
Best Guess = 0x75, Corr = 0.894498765030352
Best Guess = 0x77, Corr = 0.8874011576131537
Best Guess = 0x79, Corr = 0.8872766112055775
Best Guess = 0x87, Corr = 0.8890355947927411
Best Guess = 0x96, Corr = 0.8453476922780022
Best Guess = 0xCA, Corr = 0.8834336711981569
Best Guess = 0x1C, Corr = 0.903127475052072
Best Guess = 0x7F, Corr = 0.8662984398584177
Best Guess = 0xC5, Corr = 0.9016144489417923
Best Guess = 0x82, Corr = 0.9164989946928866
Recovered: C6 6A A6 12 4A BA 4D 04 4A 22 03 54 5B 28 0E 63


We now have everything we need to recover the full key! We'll start by combining the 13th and 14th round keys:

In [36]:
rec_key_comb = real_key2.copy()
rec_key_comb.extend(rec_key)

print("Key:", end="")
for subkey in rec_key_comb:
    print(" {:02X}".format(subkey), end="")
print("")

Key: C6 6A A6 12 4A BA 4D 04 4A 22 03 54 5B 28 0E 63 EA 79 79 20 C8 71 44 7D 46 62 5F 51 85 C1 3B CB


and then we can use the `AES128_8bit` leakage model to recover the first two rounds:

In [37]:
result = leak_model.keyScheduleRounds(rec_key_comb, 13, 0)
result.extend(leak_model.keyScheduleRounds(rec_key_comb, 13, 1))
print("Key:", end="")
for subkey in result:
    print(" {:02X}".format(subkey), end="")
print("")

Key: 94 28 5D 4D 6D CF EC 08 D8 AC DD F6 BE 25 A4 99 C4 D9 D0 1E C3 40 7E D7 D5 28 D4 09 E9 F0 88 A1


You should see a 32 byte key printed out. Open `supersecret.h`, confirm that we have the right key, and celebrate! 

## Recovering the IV

Now that we have the encryption key, we can proceed onto an attack of the next secret value: the IV.

Here, we have the luxury of seeing the source code of the bootloader. This is generally not something we would have access to in the real world, so we'll try not to use it to cheat. (Peeking at `supersecret.h` counts as cheating.) Instead, we'll use the source to help us identify important parts of the power traces.

### Bootloader Source Code

Inside the bootloader's main loop, it does three tasks that we're interested in:

* it decrypts the incoming ciphertext;
* it applies the IV to the decryption's result; and
* it checks for the signature in the resulting plaintext.

This snippet from `bootloader.c` shows all three of the tasks:

```C
// Continue with decryption
trigger_high();                
aes256_decrypt_ecb(&ctx, tmp32);
trigger_low();
             
// Apply IV (first 16 bytes)
for (i = 0; i < 16; i++){
    tmp32[i] ^= iv[i];
}

//Save IV for next time from original ciphertext                
for (i = 0; i < 16; i++){
    iv[i] = tmp32[i+16];
}

// Tell the user that the CRC check was okay
putch(COMM_OK);
putch(COMM_OK);

//Check the signature
if ((tmp32[0] == SIGNATURE1) &&
   (tmp32[1] == SIGNATURE2) &&
   (tmp32[2] == SIGNATURE3) &&
   (tmp32[3] == SIGNATURE4)){
   
   // Delay to emulate a write to flash memory
   _delay_ms(1);
}   
```

This gives us a pretty good idea of how the microcontroller is going to do its job, but if you'd like to go further, you can open the `.lss` file for the binary that was built. This is called a listing file and it lets you see the assembly that the C was compiled and linked to.

### Power Traces

As you can see from both files, after the decryption process, the bootloader executes a few distinct pieces of code:

* To apply the IV, it uses an XOR operation;
* To store the new IV, it copies the previous ciphertext into the IV array;
* It sends two bytes on the serial port;
* It checks the bytes of the signature one by one.

We should be able to recognize these four parts of the code in the power traces. Let's modify our capture routine to find them:

1. We're looking for the original IV, but it's overwritten after each successful decryption. This means we'll have to reset the target before each trace we capture
1. We'd like to skip over all of the decryption process. Recall that the trigger pin is set low after the decryption finishes. This means we can skip over the AES-256 function by triggering on a falling edge instead
1. Depending on the target, we may have to flush the target's serial lines by sending it a bunch of invalid data and looking for a bad CRC return. This slows down the capture process by a lot, so you may want to try without doing this first.
1. We won't need as many samples, so we can reduce how many we capture. 3000 should be sufficient for most targets.

Let's start by reducing our samples and making a function to reset our target (depending on your target, you may need to change the reset pin):

In [109]:
import time
scope.adc.samples = 3000
def reset_target(scope):
    scope.io.nrst = 'low'
    #scope.io.pdic = 'low'
    time.sleep(0.05)
    scope.io.nrst = 'high'
    #scope.io.pdic = 'high'

We can trigger on a falling edge by changing `scope.adc.basic_mode` to `"falling_edge"`:

In [110]:
scope.adc.basic_mode = "falling_edge"

We can flush the serial line by sending an invalid message, then checking for a bad CRC return value (`0xA1`). Let's make sure our changes work by getting a trace:

In [40]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from chipwhisperer.capture.acq_patterns.basic import AcqKeyTextPattern_Basic
reset_target(scope)
message = [0x00]


num_char = target.ser.inWaiting()
target.ser.read(num_char)

key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here

message.extend(text)

crc = bl_crc.bit_by_bit(text)
message.append(crc >> 8)
message.append(crc & 0xFF)

#flush target's serial
okay = 0
while not okay:
    target.ser.write("\0xxxxxxxxxxxxxxxxxx")
    time.sleep(0.005)
    num_char = target.ser.inWaiting()
    response = target.ser.read(num_char)
    if response:
        if ord(response[0]) == 0xA1:
            okay = 1

scope.arm()

target.ser.write(message)
timeout = 50
# wait for target to finish
while target.isDone() is False and timeout:
    timeout -= 1
    time.sleep(0.01)

try:
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')
except IOError as e:
    print('IOError: %s' % str(e))

# run aux stuff that should happen after trace here
num_char = target.ser.inWaiting()
response = target.ser.read(num_char)
if ord(response[0]) != 0xA4:
    # Bad response, just skip
    print("Bad response: {:02X}".format(ord(response[0])))


trace = scope.getLastTrace()


output_notebook()
p = figure()

xrange = range(len(trace))
p.line(xrange, trace, line_color="red")
show(p)

Loading BokehJS ...

You should see 5 different sections:

* 16 XORs
* 16 register loads (this is the new IV being copied over)
* Some serial communication
* The signature check
* The serial line going idle

Different targets have different power traces (for example, on Arm the XORs and register loads are almost identical), but hopefully you can pick out where each section is. For example, on XMEGA:

![XMEGA_Bonus_Trace](https://wiki.newae.com/images/f/f6/Tutorial-A5-Bonus-Trace-Notes.PNG)

With all of these things clearly visible, we have a pretty good idea of how to attack the IV and the signature. We should be able to look at each of the XOR spikes to find each of the IV bytes - each byte is processed on its own. Then, the signature check uses a short-circuiting comparison: as soon as it finds a byte in error, it stops checking the remaining bytes. This type of check is susceptible to a timing attack.

With those things done, we can move onto our capture loop. It's pretty similar to our last one. We're done with Analyzer, so we can store our traces in Python lists (we'll convert to numpy arrays later for easy analysis).

In [405]:
from tqdm import tqdm
from chipwhisperer.capture.acq_patterns.basic import AcqKeyTextPattern_Basic
import numpy as np
import time
traces = []
keys = []
plaintexts = []
key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here
N = 500  # Number of traces
target.init()
for i in tqdm(range(N), desc='Capturing traces'):
    reset_target(scope)
    message = [0x00]
    

    num_char = target.ser.inWaiting()
    target.ser.read(num_char)
    
    key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here
    keys.append(key)
    plaintexts.append(text)
    
    message.extend(text)
    
    crc = bl_crc.bit_by_bit(text)
    message.append(crc >> 8)
    message.append(crc & 0xFF)

    # run aux stuff that should run before the scope arms here
    
    #flush target's serial
    okay = 0
    while not okay:
        target.ser.write("\0xxxxxxxxxxxxxxxxxx")
        time.sleep(0.005)
        num_char = target.ser.inWaiting()
        response = target.ser.read(num_char)
        if response:
            if ord(response[0]) == 0xA1:
                okay = 1
    scope.arm()

    # run aux stuff that should run after the scope arms here

    target.ser.write(message)
    timeout = 50
    # wait for target to finish
    while target.isDone() is False and timeout:
        timeout -= 1
        time.sleep(0.01)

    try:
        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition')
            continue
    except IOError as e:
        print('IOError: %s' % str(e))

    # run aux stuff that should happen after trace here
    num_char = target.ser.inWaiting()
    response = target.ser.read(num_char)
    if ord(response[0]) != 0xA4:
        # Bad response, just skip
        print("Bad response: {:02X}".format(ord(response[0])))
        continue
    
    traces.append(scope.getLastTrace())


Capturing traces:   5%|▌         | 27/500 [00:19<04:31,  1.74it/s]

KeyboardInterrupt: 

### Analysis

#### Attack Theory

The bootloader applies the IV to the AES decryption result by calculating


$\text{PT} = \text{DR} \oplus \text{IV}$

where DR is the decrypted ciphertext, IV is the secret vector, and PT is the plaintext that the bootloader will use later. We only have access to one of these: since we know the AES-256 key, we can calculate DR. This exclusive or should be visible in the power traces

This is enough information for us to attack a single bit of the IV. Suppose we only wanted to get the first bit (number 0) of the IV. We could do the following:

* Split all of the traces into two groups: those with DR[0] = 0, and those with DR[0] = 1.
* Calculate the average trace for both groups.
* Find the difference between the two averages. It should include a noticeable spike during the first iteration of the loop.
* Look at the direction of the spike to decide if the IV bit is 0 `(PT[0] = DR[0])` or if the IV bit is 1 `(PT[0] = ~DR[0])`.

This is effectively a DPA attack on a single bit of the IV. We can repeat this attack 128 times to recover the entire IV.

#### A 1-Bit Attack

Recall that we're looking for the xor operation between the last decrypted block, so we'll need to decrypt it up to that point. The PyCrypto includes an AES decyprtion routine, so we'll be using that. We'll start by importing the necessary modules and converting our traces/plaintext to numpy arrays:

In [171]:
from Crypto.Cipher import AES
import numpy as np

trace_array = np.asarray(traces)  # if you prefer to work with numpy array for number crunching
textin_array = np.asarray(plaintexts)

numTraces = len(trace_array)
traceLen = len(trace_array[0])

Next we'll do the AES256 decryption. If you got a different key in the earlier part, you'll need to change `knownkey`.

In [172]:
knownkey = [0x94, 0x28, 0x5D, 0x4D, 0x6D, 0xCF, 0xEC, 0x08, 0xD8, 0xAC, 0xDD, 0xF6, 0xBE, 0x25, 0xA4, 0x99,
            0xC4, 0xD9, 0xD0, 0x1E, 0xC3, 0x40, 0x7E, 0xD7, 0xD5, 0x28, 0xD4, 0x09, 0xE9, 0xF0, 0x88, 0xA1]

knownkey = bytes(knownkey)
dr = []
aes = AES.new(knownkey, AES.MODE_ECB)
for i in range(numTraces):
    ct = bytes(textin_array[i])
    pt = aes.decrypt(ct)
    d = [bytearray(pt)[i] for i in range(16)]
    dr.append(d)

Now, let's split the traces into two groups by comparing bit 0 of the DR:

In [173]:
groupedTraces = [[] for _ in range(2)]
for i in range(numTraces):
    bit0 = dr[i][0] & 0x01
    groupedTraces[bit0].append(trace_array[i])
print(len(groupedTraces[0]))

1


If you have 1000 traces, you should expect this to print a number around 500 - roughly half of the traces should fit into each group. Now, NumPy's average function lets us easily calculate the average at each point:

In [49]:
# Find averages and differences
means = []
for i in range(2):
    means.append(np.average(groupedTraces[i], axis=0))
diff = means[1] - means[0]

Finally, we can plot this difference to see if we can spot the IV:

In [50]:
# Split traces into 2 groups
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure()

xrange = range(len(diff))
xrange2 = range(len(traces[0]))
p.line(xrange, diff, line_color="red")
#p.line(xrange2, traces[0], line_color='blue')
show(p)

Loading BokehJS ...

You should see a few visible spikes. We're looking for the XOR for byte 0 here, so any later spikes won't be the XOR. Use bokeh's zoom functionality to pinpoint all the largest spikes and record their sample location. You'll probably need to record a few: only one is the correct spike, but we won't be able to tell until we repeat this with other bytes. For example, you might have spikes at 37, 41, and 45. Make sure you record all these values. These peaks won't all be above 0, so make sure you're looking at both positive and negative values.

Next, we'll need to repeat this with a few more bytes. To make things easier, the necessary code has been combined into the below block. Increment the `0` in `bit0 = dr[i][0] & 0x01` to other numbers to attack other bytes. Attacking bytes 0 through 3 should be sufficient.

In [51]:
groupedTraces = [[] for _ in range(2)]
for i in range(numTraces):
    bit0 = dr[i][0] & 0x01
    groupedTraces[bit0].append(trace_array[i])
print(len(groupedTraces[0]))

# Find averages and differences
means = []
for i in range(2):
    means.append(np.average(groupedTraces[i], axis=0))
diff = means[1] - means[0]

# Split traces into 2 groups
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure()

xrange = range(len(diff))
xrange2 = range(len(traces[0]))
p.line(xrange, diff, line_color="red")
show(p)

497


Loading BokehJS ...

Now that you have some peak data, you'll want to use this to find the time shift between XORs. This time shift should be constant between samples and needs to work for all samples (each run through the loop is the same, so it makes sense that the time shift should be constant). For example, you might have:

```
0th byte @ 37, 41
1st byte @ 77, 81
2nd byte @ 105, 117, 121
3rd byte @ 141, 157, 161
4th byte @ 197, 201
```

With this data, peaks at 41, 81, 121, 161, and 201 have a constant time shift of 40. This means the location of the XORs is `41 + 40 * byte#`

#### The Other 127

The best way to attack the IV would be to repeat the 1-bit conceptual attack for each of the bits. Try to do this yourself! (Really!) If you're stuck, here are a few hints to get you going:

One easy way of looping through the bits is by using two nested loops, like this:

```python
for byte in range(16):
    for bit in range(8):
        # Attack bit number (byte*8 + bit)
```

The sample that you'll want to look at will depend on which byte you're attacking. We had success when we used `location = 51 + byte*60`, but your mileage will vary.

The bitshift operator and the bitwise-AND operator are useful for getting at a single bit:

```python
# This will either result in a 0 or a 1
checkIfBitSet = (byteToCheck >> bit) & 0x01
```

If you're really, really stuck, the end of this tutorial has a working script. After finding the IV, check `supersecret.h` and verify that your attack was successful.

## Attacking the Signature

Since the bootloader has different execution times based on the number of correct bytes, it's possible to perform a timing attack similar to the one covered in Tutorial B3. If you're feeling very confident in your SPA skills, you may want to attempt this yourself. This attack, however, won't be covered in this tutorial.

The last thing we can do with this bootloader is attack the signature. This final section will show how one byte of the signature could be recovered. If you want more of this kind of analysis, a more complete timing attack is shown in Tutorial B3-1 Timing Analysis with Power for Password Bypass.

### Attack Theory

Recall from earlier that the signature check in C looks like:

```C
if ((tmp32[0] == SIGNATURE1) &&
    (tmp32[1] == SIGNATURE2) &&
    (tmp32[2] == SIGNATURE3) &&
    (tmp32[3] == SIGNATURE4)){
```

Open the listing (`.lss`) file for your binary and find the above check. You should find that the first unsuccessful check will cause it to branch. In C, boolean expressions support short-circuiting. When checking multiple conditions, the program will stop evaluating these booleans as soon as it can tell what the final value will be. In this case, unless all four of the equality checks are true, the result will be false. Thus, as soon as the program finds a single false condition, it's done.

The assembly code confirms this short-circuiting operation. Each of the four assembly blocks include a comparison and a conditional branch. All four of the branches return the program to the same location (the start of the while(1) loop). If any of the comparisons are false, the branches will return the program back to the start of the loop. All four branches must fail to get into the body of the if block.

The short-circuiting conditions are perfect for us. We can use our power traces to watch how long it takes for the signature check to fail. If the check takes longer than usual, then we know that the first byte of our signature was right.

In [463]:
from tqdm import tqdm
from chipwhisperer.capture.acq_patterns.basic import AcqKeyTextPattern_Basic
import numpy as np
import time
traces = []
keys = []
plaintexts = []

iv = [0xC1, 0x25, 0x68, 0xDF, 0xE7, 0xD3, 0x19, 0xDA, 0x10, 0xE2, 0x41, 0x71, 0x33, 0xB0, 0xEB, 0x3C]

knownkey = [0x94, 0x28, 0x5D, 0x4D, 0x6D, 0xCF, 0xEC, 0x08, 0xD8, 0xAC, 0xDD, 0xF6, 0xBE, 0x25, 0xA4, 0x99,
            0xC4, 0xD9, 0xD0, 0x1E, 0xC3, 0x40, 0x7E, 0xD7, 0xD5, 0x28, 0xD4, 0x09, 0xE9, 0xF0, 0x88, 0xA1]


knownkey = bytes(knownkey)
aes = AES.new(knownkey, AES.MODE_ECB)

key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here
N = 100 # Number of traces
sig_start = 0x00
sig_start2 = 0xEB

reset_target(scope)
okay=0
scope.adc.basic_mode = "falling_edge"
while not okay:
    target.ser.write("\0xxxxxxxxxxxxxxxxxx")
    time.sleep(0.005)
    num_char = target.ser.inWaiting()
    response = target.ser.read(num_char)
    if response:
        if ord(response[0]) == 0xA1:
            okay = 1

attack_spot = 0

scope.adc.samples = 24000
scope.adc.offset = 0
target.init()
for i in tqdm(range(N), desc='Capturing traces'):
    #reset_target(scope)
    message = [0x00]
    

    num_char = target.ser.inWaiting()
    target.ser.read(num_char)
    
    key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here
    
    if not attack_spot:
        text = [0] * 16
        #text[1] = 0xEB
        #text[2] = 0x02
        #text[3] = 0x1D
        attack_spot += 1
        
    #text[0] = 0
    text2 = [0] * 16
    text2[:] = text[:]
    textcpy = [0] * 16
    textcpy[:] = text[:]
    
    #tmp = text[attack_spot]
    #text[attack_spot] = sig_start
    #sig_start = tmp
    
    #tmp = text[attack_spot+1]
    #text[attack_spot+1] = sig_start2
    #sig_start2 = tmp
    
    plaintexts.append(textcpy)
    
    for i in range(len(iv)):
        text[i] ^= iv[i]
        
    
    for i in range(16):
        print("{:02X}".format(text[i]), end="")
    
    print("")
    
    ct = aes.encrypt(bytes(text))
    pt = bytearray(aes.decrypt(ct))
    for i in range(16):
        pt[i] ^= iv[i]
        
    for i in range(16):
        print("{:02X}".format(pt[i]), end="")
    
    print("")
    message.extend(ct)
    iv[:] = ct[:]
    
    crc = bl_crc.bit_by_bit(ct)
    message.append(crc >> 8)
    message.append(crc & 0xFF)

    # run aux stuff that should run before the scope arms here
    
    #flush target's serial
    okay = 0

    scope.arm()

    # run aux stuff that should run after the scope arms here

    target.ser.write(message)
    timeout = 50
    # wait for target to finish
    while target.isDone() is False and timeout:
        timeout -= 1
        time.sleep(0.01)

    try:
        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition')
            continue
    except IOError as e:
        print('IOError: %s' % str(e))

    # run aux stuff that should happen after trace here
    num_char = target.ser.inWaiting()
    response = target.ser.read(num_char)
    if ord(response[0]) != 0xA4:
        # Bad response, just skip
        print("Bad response: {:02X}".format(ord(response[0])))
        continue
    #if "x" in response:
        #print("Got past sig")
    
    #print(response)
    #print(scope.adc.trig_count)
    
    traces.append(scope.getLastTrace())



Capturing traces:   0%|          | 0/100 [00:00<?, ?it/s]

C12568DFE7D319DA10E2417133B0EB3C
00000000000000000000000000000000




Capturing traces:   1%|          | 1/100 [00:00<00:21,  4.65it/s]

FCBA91B126B01F4AE82EEB9787044525
2B591786C4329BC178FD9FDC5CF2EECB




Capturing traces:   2%|▏         | 2/100 [00:00<00:20,  4.67it/s]

Capturing traces:   3%|▎         | 3/100 [00:00<00:19,  4.91it/s]

...

Capturing traces:  98%|█████████▊| 98/100 [00:26<00:00,  4.34it/s]

1B06C7750D8CA2EA305FBAF1B6C85CD4
A246C03C20681C64FA75D7C9A752728F




Capturing traces:  99%|█████████▉| 99/100 [00:26<00:00,  3.80it/s]

350626B0CD2485B7C6107FBF6C605DD5
27856C4D4EF484C8851937C8798422FE




Capturing traces: 100%|██████████| 100/100 [00:26<00:00,  4.00it/s]



### Finding a Single Byte

Okay, we know that our power trace will look a lot different for one of our choices of signatures. Let's figure out which one. We'll start by finding the average over all of our traces:

In [464]:
# Find the average over all of the traces
mean = np.average(traces, axis=0)
#p = figure()


#p.line(range(len(traces[0])), traces[0])
#p.line(range(len(traces[1])), traces[1], line_color="red")
#show(p)

Then, we'll split our traces into 256 different groups (one for each plaintext). Since we know the IV, we can now use it to recover the actual plaintext that the bootloader checks:

In [465]:
# Split the traces into groups
groupedTraces = [[] for _ in range(256)]
#print(plaintexts)
for i in range(N):
    group = plaintexts[i][0]
    #print(group)
    if not traces[i].all():
        groupedTraces[group].append(traces[i])
    else:
        print("Got blank trace {}".format(i))
    

Next, we can find the mean for each group and see how much they differ from the overall mean:

In [466]:
# Find the mean for each group
means = np.zeros([256, 24000])
for i in range(256):
    #print(groupedTraces[i], end="\n\n")
    if len(groupedTraces[i]) > 0:
        means[i] = np.average(groupedTraces[i], axis=0)
        
p = figure()
colors = ["red", "blue", "green", "yellow"]
xrange = range(len(means[0]))
#print(groupedTraces[0])
for i in range(0,10):
    if len(groupedTraces[i]) > 0:
        p.line(xrange, means[i]-mean, line_color=colors[i%4])
    
show(p)

In [410]:
corr = []
for i in range(256):
    corr.append(np.corrcoef(mean[18000:20000], means[i][18000:20000])[0, 1])
print(np.sort(corr))
print(np.argsort(corr))

[0.99910735 0.99916313 0.99926892 0.9992867  0.99928679 0.99929441
 0.99931374 0.99932227 0.99932935 0.99933034 0.99935479 0.99936541
 0.99936583 0.99936674 0.99936791 0.99936929 0.99937529 0.99937624
 0.99938425 0.99938568 0.99939836 0.99940096 0.99940416 0.99940434
 0.99941015 0.99941797 0.99942118 0.99942391 0.99942515 0.99942711
 0.99942725 0.99942965 0.99943058 0.99944477 0.9994455  0.99944652
 0.99944685 0.99945296 0.99945345 0.99945472 0.99946473 0.99946854
 0.99946971 0.9994704  0.99947268 0.99947716 0.99948722 0.99949253
 0.99949565 0.99949709 0.99949817 0.9994987  0.9995093  0.99950964
 0.99951831 0.99951985 0.99952205 0.99952219 0.99952297 0.9995269
 0.99952724 0.99953169 0.99953366 0.99954474 0.9995462  0.99955119
 0.9995624  0.99956565 0.99958732 0.9995994  0.99960577 0.99960835
 0.99970463 0.99972234 0.99972506 0.99974211 0.99974402 0.99975457
 0.99976496 0.99977074 0.99977344 0.99977699 0.99977984 0.99980707
 0.99984072        nan        nan        nan        nan        

/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2400: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2401: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


## Conclusion

We've now successfully recovered the encryption key and the IV for the bootloader!

## Appendix A: IV Attack Script

Make sure you've run the IV analysis blocks before running this block.

In [52]:
for byte in range(16):
    location = 41 + byte * 40
    iv = 0
    for bit in range(8):
        pt_bits = [((dr[i][byte] >> (7-bit)) & 0x01) for i in range(numTraces)]

        # Split traces into 2 groups
        groupedPoints = [[] for _ in range(2)]
        for i in range(numTraces):
            groupedPoints[pt_bits[i]].append(trace_array[i][location])
            
        means = []
        for i in range(2):
            means.append(np.average(groupedPoints[i]))
        diff = means[1] - means[0]
        
        iv_bit = 1 if diff > 0 else 0
        iv = (iv << 1) | iv_bit
        
        print(iv_bit, end = " ")
        
    print("{:02X}".format(iv))

1 1 0 0 0 0 0 1 C1
0 0 1 0 0 1 0 1 25
0 1 1 0 1 0 0 0 68
1 1 0 1 1 1 1 1 DF
1 1 1 0 0 1 1 1 E7
1 1 0 1 0 0 1 1 D3
0 0 0 1 1 0 0 1 19
1 1 0 1 1 0 1 0 DA
0 0 0 1 0 0 0 0 10
1 1 1 0 0 0 1 0 E2
0 1 0 0 0 0 0 1 41
0 1 1 1 0 0 0 1 71
0 0 1 1 0 0 1 1 33
1 0 1 1 0 0 0 0 B0
1 1 1 0 1 0 1 1 EB
0 0 1 1 1 1 0 0 3C
